In [ ]:
require 'nn'
require 'rnn'
require 'optim'

# Trying this separately

In [1]:
require 'nn'
require 'rnn'
require 'optim'

In [2]:
nepochs = 1000
learning_rate = 1e-6
gamma =0.2
cuts =4
base_explore_rate = 0.1 
delta = 1./(nepochs/cuts) 

SKIP = 1
SELECT = 2

vocabSize = 16
embeddingSize = 64

torch.manualSeed(420)
math.randomseed(420)

bow = true

if bow then
    sentenceLookup = nn.Sequential()
                :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                :add(nn.Sum(2, 3, false))
else
    sentenceLookup = nn.Sequential()
                :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                :add(nn.SplitTable(1, embeddingSize))
                :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                :add(nn.Linear(embeddingSize, embeddingSize))
                :add(nn.ReLU())
end
queryLookup = sentenceLookup:clone("weight", "gradWeight") 
summaryLookup = sentenceLookup:clone("weight", "gradWeight")

pmodule = nn.ParallelTable()
            :add(sentenceLookup)
            :add(queryLookup)
            :add(summaryLookup)

model = nn.Sequential()
        :add(pmodule)
        :add(nn.JoinTable(2))
        :add(nn.Tanh())
        :add(nn.Linear(embeddingSize * 3, 2))


criterion = nn.MSECriterion()
params, gradParams = model:getParameters()

function buildSummary(actions, sentences, buffer)
    buffer:zero()

     bufferSize = buffer:size(2)
     actionsSize = actions:size(1)
     sentencesSize = sentences:size(2)

     mask1 = torch.eq(actions:select(2,2), 1):view(actionsSize, 1):expand(
        actionsSize, sentencesSize)
     allTokens = sentences:maskedSelect(mask1)
     mask2 = torch.gt(allTokens,0)
     allTokens = allTokens:maskedSelect(mask2)

    if allTokens:dim() > 0 then
         copySize = math.min(bufferSize, allTokens:size(1))

        buffer[1]:narrow(1, bufferSize - copySize + 1, copySize):copy(
            allTokens:narrow(1, allTokens:size(1) - copySize + 1, copySize))
    end
    return buffer
end

function buildTokenCounts(summary)
    counts = {}
    for i=1,summary:size(2) do
        if summary[1][i] > 0 then
             token = summary[1][i]
            if counts[token] == nil then
                counts[token] = 1
            else
                counts[token] = counts[token] + 1
            end
        end
    end
    return counts
end

function rougeScores(genSummary, refSummary)
     genTotal = 0
     refTotal = 0
     intersection = 0
    for k, refCount in pairs(refSummary) do
         genCount = genSummary[k]
        if genCount == nil then genCount = 0 end
        intersection = intersection + math.min(refCount, genCount)
        refTotal = refTotal + refCount
    end
    for k,genCount in pairs(genSummary) do
        genTotal = genTotal + genCount
    end

    if genTotal == 0 then 
        genTotal = 1 
    end
     recall = intersection / refTotal
     prec = intersection / genTotal
    if recall > 0 and prec > 0 then
        f1 = 2 * recall * prec / (recall + prec)
    else 
        f1 = 0
    end
    return recall, prec, f1
end

function backProp(input_memory, params, model, criterion, batch_size, memsize)
    -- local input = input_memory[1]
    -- local reward = input_memory[2]
    local inputs = {input_memory[1], input_memory[3]}
    local rewards = input_memory[2]
    local dataloader = dl.TensorLoader(inputs, rewards)
    local err = 0.    

    den = 1
    for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
        xinput = xin[1]
        actions_in = xin[2]
        local function feval(params)
            gradParams:zero()
            local predQ = model:forward(xinput)
            local maskLayer = nn.MaskedSelect()
            local predQOnActions = maskLayer:forward({predQ, actions_in})

            local lossf = criterion:forward(predQOnActions, reward)
            local gradOutput = criterion(predQOnActions, reward)
            local gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput)
            model:backward(xinput, gradMaskLayer[1])
            return lossf, gradParams
        end
        --- This is confusing to me...
        _, lossv  = optim.rmsprop(feval, params, optimParams)   
    end
    return lossv[1]
end

In [3]:
optimParams = {
    learningRate = learning_rate,
}

maxSummarySize = 36
epsilon = 1.0
query = torch.LongTensor{{0, 1, 4, 3}}
sentenceStream = torch.Tensor{{0, 1, 3, 4}, 
                                {7, 6, 5 ,8}, 
                                {0, 2, 4, 3}, 
                                {7, 5, 8, 6}, 
                                {1, 4, 3, 2}, 
                                {13, 14, 15, 16}}

refSummary = torch.Tensor{{1,3,4,2,4,3,1,4,3,2,9,10,12,11}}
refCounts = buildTokenCounts(refSummary)


streamSize = sentenceStream:size(1)
bestActions = torch.ByteTensor{{0,1},{1,0},{0,1},{1,0},{0,1},{1,0}}


buffer = torch.Tensor(1, maxSummarySize):zero()
bestSummary = buildSummary(
    bestActions:narrow(1, 1, 6), 
    sentenceStream:narrow(1, 1, 6),
    buffer:narrow(1, 1, 1)
    )

generatedCounts = buildTokenCounts(bestSummary) 
bestrecall, bestprec, bestf1 = rougeScores(generatedCounts, refCounts)
print(string.format("TRUE {RECALL = %.6f, PREC = %.6f, F1 = %.6f}", bestrecall, bestprec, bestf1))

TRUE {RECALL = 0.714286, PREC = 1.000000, F1 = 0.833333}	


In [4]:
actions = torch.ByteTensor(streamSize,2):fill(0)
exploreDraws = torch.Tensor(streamSize)
summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
qValues = torch.Tensor(streamSize, 2):zero()
rouge = torch.Tensor(streamSize + 1):zero()

rouge[1] = 1
exploreDraws:uniform(0, 1)
summary = summaryBuffer:zero():narrow(1,1,1)

In [5]:
i = 1

In [6]:
sentence = sentenceStream:narrow(1, i, 1)

In [64]:
sentenceLookup = nn.Sequential()
            :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
            :add(nn.Sum(2, 3, false))

sentenceLookup = nn.Sequential()
            :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
            :add(nn.SplitTable(2))
            :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
            :add(nn.SelectTable(-1))

queryLookup = sentenceLookup:clone("weight", "gradWeight") 
summaryLookup = sentenceLookup:clone("weight", "gradWeight")

pmodule = nn.ParallelTable()
            :add(sentenceLookup)
            :add(queryLookup)
            :add(summaryLookup)

model = nn.Sequential()
        :add(pmodule)
        :add(nn.JoinTable(2))
        :add(nn.Tanh())
        :add(nn.Linear(embeddingSize * 3, 2))

-- tmp1 = pmodule:forward({sentence, query, summary})

In [65]:
tmp = nn.Sequential():add(nn.LookupTableMaskZero(vocabSize, embeddingSize))

In [66]:
#tmp:forward(sentence)

  1
  4
 64
[torch.LongStorage of size 3]



In [67]:
#sentenceLookup:forward(sentence)

  1
 64
[torch.LongStorage of size 2]



In [68]:
#sentenceLookup:forward(summary)

  1
 64
[torch.LongStorage of size 2]



In [56]:
pmodule:forward({sentence, query, summary})

{
  1 : DoubleTensor - size: 4x64
  2 : DoubleTensor - size: 4x64
  3 : DoubleTensor - size: 36x64
}


In [47]:
#nn.Linear(embeddingSize * 3, 2):forward( nn.JoinTable(1):forward(tmp1))

 2
[torch.LongStorage of size 1]



In [48]:
#model:forward({sentence, query, summary})

...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:67: 
In 2 module of nn.Sequential:
...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:39: inconsistent tensor size at /Users/franciscojavierarceo/torch/pkg/torch/lib/TH/generic/THTensorCopy.c:7
stack traceback:
	[C]: in function 'copy'
	...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:39: in function <...javierarceo/torch/install/share/lua/5.1/nn/JoinTable.lua:21>
	[C]: in function 'xpcall'
	...javierarceo/torch/install/share/lua/5.1/nn/Container.lua:63: in function 'rethrowErrors'
	...avierarceo/torch/install/share/lua/5.1/nn/Sequential.lua:44: in function 'forward'
	[string "local f = function() return #model:forward({s..."]:1: in function 'f'
	[string "local f = function() return #model:forward({s..."]:1: in main chunk
	[C]: in function 'xpcall'
	...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:210: in function <...ojavierarceo/torch/install/share/lua/5.1/itorch/main.lua:174>
	...ojavierarceo/

In [ ]:
qValues[i]:copy(model:forward({sentence, query, summary}))

if qValues[i][SKIP] > qValues[i][SELECT] then
    actions[i][SKIP] = 1
else
    actions[i][SELECT] = 1
end

summary = buildSummary(
    actions:narrow(1, 1, i), 
    sentenceStream:narrow(1, 1, i),
    summaryBuffer:narrow(1, i + 1, 1)
    )
generatedCounts = buildTokenCounts(summary) 
recall, prec, f1 = rougeScores(generatedCounts, refCounts)
rouge[i + 1] = f1

In [ ]:
for i=1, streamSize do
    --- the i extracts individual sentences from the stream
    sentence = sentenceStream:narrow(1, i, 1)
    qValues[i]:copy(model:forward({sentence, query, summary}))

    if exploreDraws[i] <= epsilon then
        actions[i][torch.random(SKIP, SELECT)] = 1
    else 
        if qValues[i][SKIP] > qValues[i][SELECT] then
            actions[i][SKIP] = 1
        else
            actions[i][SELECT] = 1
        end
    end

    summary = buildSummary(
        actions:narrow(1, 1, i), 
        sentenceStream:narrow(1, 1, i),
        summaryBuffer:narrow(1, i + 1, 1)
        )

    generatedCounts = buildTokenCounts(summary) 
    recall, prec, f1 = rougeScores(generatedCounts, refCounts)
    rouge[i + 1] = f1
end

In [ ]:
tmp0 = {}
tmp1 = {}
tmp2 = {}
for epoch=1,nepochs do
    actions = torch.ByteTensor(streamSize,2):fill(0)
    exploreDraws = torch.Tensor(streamSize)
    summaryBuffer = torch.LongTensor(streamSize + 1, maxSummarySize):zero()
    qValues = torch.Tensor(streamSize, 2):zero()
    rouge = torch.Tensor(streamSize + 1):zero()

    rouge[1] = 1
    exploreDraws:uniform(0, 1)

    summary = summaryBuffer:zero():narrow(1,1,1)
    for i=1, streamSize do
        --- the i extracts individual sentences from the stream
         sentence = sentenceStream:narrow(1, i, 1)
        qValues[i]:copy(model:forward({sentence, query, summary}))

        if exploreDraws[i] <= epsilon then
            actions[i][torch.random(SKIP, SELECT)] = 1
        else 
            if qValues[i][SKIP] > qValues[i][SELECT] then
                actions[i][SKIP] = 1
            else
                actions[i][SELECT] = 1
            end
        end

        summary = buildSummary(
            actions:narrow(1, 1, i), 
            sentenceStream:narrow(1, 1, i),
            summaryBuffer:narrow(1, i + 1, 1)
            )

         generatedCounts = buildTokenCounts(summary) 
         recall, prec, f1 = rougeScores(generatedCounts, refCounts)
        rouge[i + 1] = f1
    end

    local max, argmax = torch.max(qValues, 2)
    local reward0 = rouge:narrow(1,2, streamSize) - rouge:narrow(1,1, streamSize)
    local reward_tp1 = gamma * reward0:narrow(1, 2, streamSize - 1):resize(streamSize)
    --- occasionally the zeros result in a nan, which is strange
    reward_tp1[reward_tp1:ne(reward_tp1)] = 0
    reward_tp1 = torch.clamp(reward_tp1, -1, 1)
    local reward = reward0 + reward_tp1
    -- print(epoch, reward0:sum(), reward_tp1:sum(), reward:sum())
    
    local querySize = query:size(2)
    local summaryBatch = summaryBuffer:narrow(1, 1, streamSize)
    local queryBatch = query:view(1, querySize):expand(streamSize, querySize) 

    local input = {sentenceStream, queryBatch, summaryBatch}
    --- Storing the data
    memory = {input, reward, actions}

    if epoch == 1 then
        fullmemory = memory 
    else
        tmp = buildMemory(memory, fullmemory, mem_size, batch_size)
        fullmemory = tmp
    end
    --- Running backprop
    if(epoch > n_rand) then 
        loss = backProp(memory, params, model, criterion, batch_size, mem_size)
    else 
        loss = {0.}
    end
    
    if epoch == 1 then
        print(input)
    end

     _, loss = optim.adam(feval, params, optimParams)
    out = string.format("%i; %.3f;%.6f;%.6f; {min=%.3f, max=%.3f}; {min=%.3f, max=%.3f}; {%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i,%i}\n", 
            epoch, epsilon, loss[1], rouge[streamSize + 1],
            reward:min(), reward:max(),
            qValues:min(), qValues:max(),
            actions[1][1], 
            actions[1][2], 
            actions[2][1], 
            actions[2][2], 
            actions[3][1], 
            actions[3][2], 
            actions[4][1],
            actions[4][2],
            actions[5][1], 
            actions[5][2], 
            actions[6][1], 
            actions[6][2] 
        )
    print(out)

    if (epsilon - delta) <= base_explore_rate then
        epsilon = base_explore_rate
    else 
        epsilon = epsilon - delta
    end
end


In [ ]:
torch.clamp(tmp0[360], -0.7, 0.7)

In [ ]:
tmp1[360]

In [ ]:
tmp1[1]

In [ ]:
nn.Threshold(0.06, 0):forward(tmp1[1])

In [ ]:
nn.Threshold(-.05, 0):forward(tmp0[360])

In [ ]:
tmp1[360]

In [ ]:
tmp2[360]

In [ ]:
tmp1[6][tmp1[6]:ne(tmp1[6])] = 0

In [ ]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp0[6] + gamma * tmp0[6]:narrow(1, 2, streamSize-1):resize(streamSize)

In [ ]:
tmp1[6]

In [ ]:
tmp0[242]

In [ ]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.7
out = 99
print(nn.Threshold(thresh, out):forward(tmp0[242]))

In [ ]:
--- If tmp0[242] < thresh, out else tmp0[242]
thresh = -0.25
out = 99
print(nn.Threshold(thresh, out):forward(-tmp0[242]))

In [ ]:
nn.Threshold(0.01, -0.001):forward(tmp0[242])

In [ ]:
nn.Threshold(0.1, 0.01):forward(nn.Threshold(-0.01, -0.01):forward(tmp0[242]))

In [ ]:
for i = 1, 500 do
    print(tmp0[i] + tmp1[i])
end

In [ ]:
sentenceStream

In [ ]:
tst = torch.cat(sentenceStream, sentenceStream, 1)

In [ ]:
tst

In [ ]:
local dl = require 'dataload'

inputs = torch.range(1,5)
targets = torch.range(1,5)

dataloader = dl.TensorLoader(inputs, targets)

local i = 1
for k, inputs, targets in dataloader:sampleiter(2,6) do
   print(string.format("batch %d, nsampled = %d; inputs = %.i", i, k, inputs[1]))
--    print(string.format("inputs:\n%stargets:\n%s", inputs, targets))
   i = i + 1
end

In [ ]:
inputs = torch.randn(5, 1)
targets = torch.LongTensor(10):random(1,10):resize(5,2)
dataloader = dl.TensorLoader(inputs, targets)

In [ ]:
i = 1
for k, input, target in dataloader:sampleiter(5, 20) do
    print(string.format("batch %d, nsampled = %d; target {i, i}", i, k))
    print(target)
   i = i + 1
end

In [ ]:
dl = require 'dataload'

inputs = torch.LongTensor({1,2, 3, 5}, {1, 2, 3}, {4, 5, 7}, {7, 8 ,8, 3, 2}, {1,2,3,4})
targets = torch.LongTensor({1,2, 3, 5}, {1, 2, 3}, {4, 5, 7}, {7, 8 ,8, 3, 2}, {1,2,3,4})
actions = torch.round(torch.rand(5))

In [ ]:
dataloader = dl.TensorLoader(inputs, targets)

In [ ]:
i = 1
for k, input, target, action in dataloader:sampleiter(1,5) do
   print(string.format("batch %d, nsampled = %d", i, k))
   i = i + 1
    if i==5 then
        print(input, target, action)
    end
end

# Getting model right

In [ ]:
require 'rnn'
require 'nn'

In [ ]:
vocabSize = 4
embeddingSize = 10

In [ ]:
sentenceLookup = nn.Sequential()
            :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
            :add(nn.Sum(2, 3, false))

In [ ]:
sentenceLookup = nn.Sequential()
                :add(nn.LookupTableMaskZero(vocabSize, embeddingSize))
                :add(nn.SplitTable(1, embeddingSize))
                :add(nn.Sequencer(nn.LSTM(embeddingSize, embeddingSize)))
                :add(nn.SelectTable(-1))            -- selects last state of the LSTM
                :add(nn.Linear(embeddingSize, embeddingSize))
                :add(nn.ReLU())

In [ ]:
queryLookup = sentenceLookup:clone("weight", "gradWeight") 
summaryLookup = sentenceLookup:clone("weight", "gradWeight")

In [ ]:
pmod = nn.ParallelTable()
        :add(sentenceLookup)
        :add(queryLookup)
        :add(summaryLookup)

In [ ]:
model = nn.Sequential()
        :add(pmod)
        :add(nn.JoinTable(2))
        :add(nn.Tanh())
        :add(nn.Linear(embeddingSize * 3, 2))

In [ ]:
s = torch.LongTensor({{1, 2, 3,4}})
q = torch.LongTensor({{2, 3, 4, 1}})
cs = torch.LongTensor({{0, 1 ,3 , 2}})
input = {s,q,cs}
input2 = {s, q}

In [ ]:
sentenceLookup:forward(cs)

In [ ]:
sentenceLookup:forward(s)

In [ ]:
nn.JoinTable(1):forward(pmod:forward(input))

In [ ]:
pmod:forward(input)

In [ ]:
model:forward(input)

In [ ]:
predQ = model:forward(input)

In [ ]:
maskLayer = nn.MaskedSelect()

In [ ]:
predQ

In [ ]:
torch.max(predQ, 2)

In [ ]:
predaction , actions = torch.max(predQ, 2)

In [ ]:
maskLayer = nn.MaskedSelect()

In [ ]:
actions

In [ ]:
predQ

In [ ]:
actions_in = torch.ByteTensor(6, 2):fill(0)

In [ ]:
predQOnActions = maskLayer:forward({predQ, actions_in})

In [ ]:
function backProp(input_memory, params, model, criterion, batch_size, memsize)
    -- local input = input_memory[1]
    -- local reward = input_memory[2]
    local inputs = {input_memory[1], input_memory[3]}
    local rewards = input_memory[2]
    local dataloader = dl.TensorLoader(inputs, rewards)
    local err = 0.    

    den = 1
    for k, xin, reward in dataloader:sampleiter(batch_size, memsize) do
        xinput = xin[1]
        actions_in = xin[2]
        local function feval(params)
            gradParams:zero()
            local predQ = model:forward(xinput)
            local maskLayer = nn.MaskedSelect()
            local predQOnActions = maskLayer:forward({predQ, actions_in})

            local lossf = criterion:forward(predQOnActions, reward)
            local gradOutput = criterion(predQOnActions, reward)
            local gradMaskLayer = maskLayer:backward({predQ, actions_in}, gradOutput)
            model:backward(xinput, gradMaskLayer[1])
            return lossf, gradParams
        end
        --- This is confusing to me...
        _, lossv  = optim.rmsprop(feval, params, optimParams)   
    end
    return lossv[1]
end